In [2]:
%load_ext autoreload
%autoreload 2
import numpy as np
import xarray as xr
from matplotlib import pyplot as plt
import pandas as pd
import sys
sys.path.append('/home/onno/Thesis/Scripts')
import my_tools
from my_tools import plot_dic, file_dic
from mpl_toolkits.basemap import Basemap, addcyclic
%matplotlib qt

In [3]:
path = '/media/onno/Algemeen/Thesis/GFS_T850/'

In [23]:
#Anomalies from 10th and 90th percentile have already been calculated using cdo
#Specify coordinates
coordinatez = [
(54,46,6,14),#Germany
(44,36,352,360), #Spain
(54,46,26,34), #Ukraine
(58,50,352,360), #UK
(42,34,28,36), #Turkey
(68,60,22,30), #Finland
(66,58,6,14), #Norway/Sweden
(60,52,46,54)] #Russia Samara/Kazan region

for lat1,lat0,lon0,lon1 in coordinatez:
    file_anom_p10 = 'era51_fldmean_mars_t850_79-19_24hourly_lon_{}_{}_lat_{}_{}_anom_from_smoothed_p10.nc'.format(lon0,lon1-2,lat1-2,lat0)
    file_anom_p90 = 'era51_fldmean_mars_t850_79-19_24hourly_lon_{}_{}_lat_{}_{}_anom_from_smoothed_p90.nc'.format(lon0,lon1-2,lat1-2,lat0)
    #Open calculated anomally files
    anom_p10 = xr.open_dataset(path+file_anom_p10).squeeze()
    anom_p90 = xr.open_dataset(path+file_anom_p90).squeeze()
    #Create numpy array for start and end dates of persistent temperature extremes
    #see my_tools.detect_heatwaves/my_tools.detect_coldwaves for further documentation
    persistent_heatwaves,short_heatwaves = my_tools.detect_heatwaves(anom_p90)
    persistent_coldwaves,short_coldwaves = my_tools.detect_coldwaves(anom_p10)
    #This cell checks whether individual short lived extremes are independent from other short lived extreme events. 
    #This is done by checking whether another short lived extreme occurs within three days
    #The result is a boolean array where independent events are labeled True and events that are not independent are labeled False
    variablez =[short_heatwaves,short_coldwaves]
    #loop over heat and coldwaves
    for variable in variablez:
        #create empty boolean array
        independent = np.zeros(len(variable),dtype=bool)
        #Caluclate differenc ein days between individual events with for loop
        for i in range(len(variable)):
            #Make for loop work for first event
            if i==0:
                after = pd.Timedelta(variable[i+1,0]-variable[i,1]).days
                if after > 3:
                    independent[i]=True
                else:
                    independent[i]=False
            #Make for loop work for last event
            elif i==len(variable)-1:
                before = pd.Timedelta(variable[i,0]-variable[i-1,1]).days
                if before > 3:
                    independent[i]=True
                else:
                    independent[i]=False
            #general calculation
            else:
                before = pd.Timedelta(variable[i,0]-variable[i-1,1]).days
                after = pd.Timedelta(variable[i+1,0]-variable[i,1]).days
                #Take shortest day difference
                combined = min(before,after)
                if combined > 3:
                    independent[i]=True
                else:
                    independent[i]=False
        #save as numpy array
        if np.all(variable == variablez[0]):
            np.save(path+'independent_short_heatwaves_lon_{}_{}_lat_{}_{}.npy'.format(lon0,lon1-2,lat1-2,lat0),independent)
        else:
            np.save(path+'independent_short_coldwaves_lon_{}_{}_lat_{}_{}.npy'.format(lon0,lon1-2,lat1-2,lat0),independent)

    #save dates as text file and numpy array
    np.savetxt(path+'persistent_heatwaves_lon_{}_{}_lat_{}_{}.txt'.format(lon0,lon1-2,lat1-2,lat0),persistent_heatwaves,fmt='%s')
    np.savetxt(path+'short_heatwaves_lon_{}_{}_lat_{}_{}.txt'.format(lon0,lon1-2,lat1-2,lat0),short_heatwaves,fmt='%s')
    np.savetxt(path+'persistent_coldwaves_lon_{}_{}_lat_{}_{}.txt'.format(lon0,lon1-2,lat1-2,lat0),persistent_coldwaves,fmt='%s')
    np.savetxt(path+'short_coldwaves_lon_{}_{}_lat_{}_{}.txt'.format(lon0,lon1-2,lat1-2,lat0),short_coldwaves,fmt='%s')
    np.save(path+'persistent_heatwaves_lon_{}_{}_lat_{}_{}.npy'.format(lon0,lon1-2,lat1-2,lat0),persistent_heatwaves)
    np.save(path+'persistent_coldwaves_lon_{}_{}_lat_{}_{}.npy'.format(lon0,lon1-2,lat1-2,lat0),persistent_coldwaves)
    np.save(path+'short_heatwaves_lon_{}_{}_lat_{}_{}.npy'.format(lon0,lon1-2,lat1-2,lat0),short_heatwaves)
    np.save(path+'short_coldwaves_lon_{}_{}_lat_{}_{}.npy'.format(lon0,lon1-2,lat1-2,lat0),short_coldwaves)


/home/onno/miniconda3/envs/thesis/lib/python3.7/site-packages/ipykernel_launcher.py:58: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
/home/onno/miniconda3/envs/thesis/lib/python3.7/site-packages/ipykernel_launcher.py:58: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
/home/onno/miniconda3/envs/thesis/lib/python3.7/site-packages/ipykernel_launcher.py:58: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
/home/onno/miniconda3/envs/thesis/lib/python3.7/site-packages/ipykernel_launcher.py:58: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
/home/onno/miniconda3/envs/thesis/lib/python3.7/site-packages/ipykernel_launcher.py:58: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
/home/onno/miniconda3/envs/thesis/lib/python3.7/site-packages/ipykernel_launcher.py:58: DeprecationWarnin